<a href="https://colab.research.google.com/github/ak2742/mlplay/blob/Fine-Tuning/11)_neo4j_graph_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install libraries

!pip install -q langchain==0.2.1 langchain_community
!pip install -q google-generativeai langchain-google-genai

!pip install -q neo4j
!pip install -q gradio

In [ ]:
#@title Create llm

from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_KEY')

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, convert_system_message_to_human=True)

In [ ]:
#@title Create chain on Graph

from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph

# You can create a free sandbox at https://sandbox.neo4j.com/
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USERNAME = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

In [ ]:
#@title Add Gradio UI

import gradio as gr

def gradio_fn(msg):
    response = chain.invoke({'query': msg})
    return response['result']

gr.Interface(
    fn=gradio_fn,
    inputs=["text"],
    outputs=["text"],
    theme="soft"
    ).launch(share=True, debug=True)